In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import itertools
import os

### Scrape One Player

In [ ]:
def scrape_player(URL: str):
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = URL[-1]
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        # player_bio["games"] = game_l

        a = soup.find("table", class_="table").findAll("tr")
        a2 = a[1:]
        # print(a[1:])
        # print(len(a[1:]))
        list_a = [b.has_attr("class") for b in a[1:]]
        print(list_a)
        list_b = np.nonzero(list_a)[0]
        print(list(itertools.pairwise(list_b)))
        print(len(list(itertools.pairwise(list_b))) + 1)
        list_c = []
        for i in list(itertools.pairwise(list_b)):
            # print(a2[i[0]:i[1]])
            list_c.append(a2[i[0] : i[1]])
        # print(a2[list_b[-1]:])
        list_c.append(a2[list_b[-1] :])
        len(list_c)
        game_list = []
        for game in list_c:
            # print(game[0])
            # print([i.text.replace("\n", "") for i in game[0].findAll("td")[:3]])
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                )
            )
            # print(dict_cols)
            events = []
            for event in game[1:]:
                # print([j.text.replace("\n", "") for j in event.findAll("td")[1:-2]])
                dict_event = dict(
                    zip(
                        ["event", "team", "position", "medal"],
                        [j.text.replace("\n", "") for j in event.findAll("td")[1:-2]],
                    )
                )
                events.append(dict_event)
            # print(events)
            dict_cols["Events"] = events
            # print(dict_cols)
            game_list.append(dict_cols)
        # print(game_list)
        player_bio["games"] = game_list

    return player_bio

In [ ]:
scrape_player("https://www.olympedia.org/athletes/1")

In [ ]:
scrape_player("https://www.olympedia.org/athletes/93860")

### Scrape All Olympic Games Participants

In [ ]:
players = []
for i in range(1, 101):
    URL = f"https://www.olympedia.org/athletes/{i}"
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = i
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        player_bio["games"] = game_l 
 
        players.append(player_bio)

with open("players.json", "w") as f:
    json.dump(players, f)

Problem: There are about 150 000 players to scrape all this data we need to split it into threads but also send these threads to different computers

### Scrape only players who won medals

We need selenium to disable the limit of displayed players and to get all medal havers

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [3]:
# driver = webdriver.Firefox()
# driver.get("https://www.olympedia.org/statistics/medal/athlete")

In [3]:
def get_medal_havers():
    URL = "https://www.olympedia.org/statistics/medal/athlete"
    page2 = requests.get(URL)

    soup2 = BeautifulSoup(page2.content, "html.parser")
    table = soup2.find("table", class_="table")

    col_names = [i.text for i in table.find("thead").findAll("th")]
    # print(col_names)

    # create webdriver object
    driver = webdriver.Firefox()
    driver.get("https://www.olympedia.org/statistics/medal/athlete")

    x = driver.find_element(By.NAME, "limit")
    drop = Select(x)

    # select by visible text
    drop.select_by_visible_text("Show all")
    # wait for all the players to be displayed on the site
    time.sleep(20)

    players = driver.find_elements(By.CLASS_NAME, "top")
    out = [i.get_attribute("innerHTML") for i in players]

    driver.close()

    players_medal = []
    for line in out:
        soup = BeautifulSoup(line, "html.parser")
        players_medal.append([i.text.lstrip() for i in soup.findAll("td")])

    df = pd.DataFrame(players_medal, columns=col_names)
    df.to_csv("data_csv/medal_havers.csv", index=False)

    return df

In [4]:
get_medal_havers()

,Athlete,Nat,Gold,Silver,Bronze,Total
0,Michael Phelps,USA,23,3,2,28
1,Larisa Latynina,URS UKR,9,5,4,18
2,Paavo Nurmi,FIN,9,3,0,12
3,Mark Spitz,USA,9,1,1,11
4,Carl Lewis,USA,9,1,0,10
...,...,...,...,...,...,...
1083,Vinzenz Geiger,GER,2,1,0,3
1084,Hanna Öberg,SWE,2,1,0,3
1085,Eileen Gu,CHN,2,1,0,3
1086,Daiki Hashimoto,JPN,2,1,0,3


## Try Threading to get all the players

In [3]:
import threading
import logging

In [ ]:
def scrape_players(r: int):
    logging.info(f"Starting Thread in range {r} - {r + 100}")
    players = []
    for i in range(r, r + 100):
        URL = f"https://www.olympedia.org/athletes/{str(i)}"
        # print(URL)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find("div", class_="athlete_bio")

        if results != None:
            table = results.find("table", class_="biodata")
            player_bio = {}
            player_bio["id"] = i
            if results.find("img", class_="photo"):
                player_bio["photo_link"] = results.find("img", class_="photo")["src"]
            for player in table.findAll("tr"):
                player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                    "td"
                ).text.lstrip()

            games = soup.find("table", class_="table").findAll("tr")
            a2 = games[1:]
            list_a = [b.has_attr("class") for b in a2]
            list_b = np.nonzero(list_a)[0]

            list_c = []
            for i in list(itertools.pairwise(list_b)):
                list_c.append(a2[i[0] : i[1]])
            list_c.append(a2[list_b[-1] :])

            game_list = []
            for game in list_c:
                dict_cols = dict(
                    zip(
                        ["game", "discipline", "noc"],
                        [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                    )
                )
                events = []
                for event in game[1:]:
                    dict_event = dict(
                        zip(
                            ["event", "team", "position", "medal"],
                            [
                                j.text.replace("\n", "")
                                for j in event.findAll("td")[1:-2]
                            ],
                        )
                    )
                    events.append(dict_event)
                dict_cols["Events"] = events
                game_list.append(dict_cols)
            player_bio["games"] = game_list

            print(player_bio)

            players.append(player_bio)
    with open(f"players/players_{r}.json", "w") as f:
        json.dump(players, f)
    logging.info(f"Thread finished {r} - {r + 100}")
    return players

Inside each batch there is 1000 players and there are 150 batches, which is a lot and must be optimized

In [ ]:
def thread_func(batch):
    thread_list = []
    for i in batch:
        thread = threading.Thread(target=scrape_players, args=(i,))
        thread_list.append(thread)
        thread.start()
    for t in thread_list:
        t.join()

In [ ]:
batches = []
for i in list(range(1, 150000, 1000)):
    batches.append([i] + [y * 100 + i for y in range(1, 10)])

In [ ]:
print(batches[1:])

In [ ]:
# for batch in batches:
#     thread_func(batch)

### Create CSV file 

Après avoir faut tourner le programme d'extraction d'informations des joueurs sur chaque page, on va le concatener. Pour faire cela, on va lister tous les fichiers JSON disponibles dans le dossier "players" 

In [64]:
# Il faut les triller, car os.listdir ne va pas les afficher dans l'ordre.
sorted_files = ["players_" + str(y) + ".json" for y in sorted([int(x.replace("players_", "").replace(".json", "")) for x in os.listdir("players/")])]

In [69]:
sorted_files[-15:]

['players_148501.json',
 'players_148601.json',
 'players_148701.json',
 'players_148801.json',
 'players_148901.json',
 'players_149001.json',
 'players_149101.json',
 'players_149201.json',
 'players_149301.json',
 'players_149401.json',
 'players_149501.json',
 'players_149601.json',
 'players_149701.json',
 'players_149801.json',
 'players_149901.json']

In [ ]:
sorted_files[:2]

On concatène les JSON.

In [117]:
python_objects = []

# Load each JSON file into a Python object.
for json_file in sorted_files[-14:]:
    with open(f"players/{json_file}", "r") as f:
        for player in json.load(f):
            python_objects.append(player)

In [118]:
python_objects

[{'id': 148601,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148601.jpg',
  'Roles': 'Competed in Olympic Games',
  'Sex': 'Male',
  'Full_name': 'Kim•Dong-Uk',
  'Used_name': 'Kim•Dong-Uk',
  'Name_order': 'Oriental',
  'Original_name': '김•동욱',
  'Other_names': 'Kim Dongwook',
  'Born': '23 April 1993',
  'NOC': 'Republic of Korea',
  'games': [{'game': '2022 Winter Olympics',
    'discipline': 'Short Track Speed Skating (Skating)',
    'noc': 'KOR'}]},
 {'id': 148602,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148602.jpg',
  'Roles': 'Competed in Olympic Games',
  'Sex': 'Male',
  'Full_name': 'Lee•Jun-Seo',
  'Used_name': 'Lee•Jun-Seo',
  'Name_order': 'Oriental',
  'Original_name': '이•준서',
  'Other_names': 'Lee June-Seo',
  'Born': '3 June 2000',
  'NOC': 'Republic of Korea',
  'games': [{'game': '2022 Winter Olympics',
    'discipline': 'Short Track Speed Skating (Skating)',
    'noc': 'KOR'}]},
 {'id': 148603,
  'photo_link': 'https://d2a3o6pzho379u.cloudfro

In [119]:
len(python_objects)

605

In [120]:
with open(f"all_players_2.json", "w") as f:
    json.dump(python_objects, f)

In [121]:
with open("all_players_2.json", 'r') as f:
    data = json.load(f)

In [122]:
df_players = pd.read_json("all_players_2.json")
df_players

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,NOC,games,Affiliations,Measurements,Nick/petnames,Died
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,Republic of Korea,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN
1,148602,https://d2a3o6pzho379u.cloudfront.net/148602.jpg,Competed in Olympic Games,Male,Lee•Jun-Seo,Lee•Jun-Seo,Oriental,이•준서,Lee June-Seo,3 June 2000,Republic of Korea,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN
2,148603,https://d2a3o6pzho379u.cloudfront.net/148603.jpg,Competed in Olympic Games,Male,Park•Jang-Hyeok,Park•Jang-Hyeok,Oriental,박•장혁,Park Jang-Hyuk,31 October 1998,Republic of Korea,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN
3,148604,NaN,Competed in Olympic Games • Other,Female,Kiana•Kryeziu,Kiana•Kryeziu,NaN,NaN,NaN,11 November 2004,Kosovo,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN
4,148605,https://d2a3o6pzho379u.cloudfront.net/148605.jpg,Competed in Olympic Games • Other,Male,Fayik•Abdi,Fayik•Abdi,NaN,فايق•عابدي,NaN,"7 October 1997 in San Diego, California (USA)",Kingdom of Saudi Arabia,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,149222,NaN,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,NaN,NaN,NaN,"30 January 2002 in Serov, Sverdlovsk (RUS)",ROC,"[{'game': '2022 Winter Olympics', 'discipline'...","Agidel, Ufa (RUS)",167 cm / 61 kg,NaN,NaN
601,149223,NaN,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,NaN,Валерия Сергеевна•Меркушева,NaN,"20 September 1999 in Moskva (Moscow), Moskva (...",ROC,"[{'game': '2022 Winter Olympics', 'discipline'...","SKIF Nizhny Novgorod, Nizhny Novgorod (RUS)",168 cm / 65 kg,NaN,NaN
602,149224,NaN,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,NaN,Юлия•Смирнова,NaN,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",ROC,"[{'game': '2022 Winter Olympics', 'discipline'...","Agidel, Ufa (RUS)",163 cm / 55 kg,NaN,NaN
603,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,NaN,NaN,NaN,"19 May 1899 in Niort, Deux-Sèvres (FRA)",France,"[{'game': '1924 Summer Olympics', 'discipline'...","Beauvoisine FC, Rouen (FRA)",166 cm,NaN,"18 March 1986 in Niort, Deux-Sèvres (FRA)"


In [84]:
df_players[df_players["id"] == 148601]

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,NOC,games,Affiliations,Measurements,Nick/petnames,Died
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,Republic of Korea,"[{'game': '2022 Winter Olympics', 'discipline'...",NaN,NaN,NaN,NaN


In [60]:
data[144874]

{'id': 148601,
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148601.jpg',
 'Roles': 'Competed in Olympic Games',
 'Sex': 'Male',
 'Full_name': 'Kim•Dong-Uk',
 'Used_name': 'Kim•Dong-Uk',
 'Name_order': 'Oriental',
 'Original_name': '김•동욱',
 'Other_names': 'Kim Dongwook',
 'Born': '23 April 1993',
 'NOC': 'Republic of Korea',
 'games': [{'game': '2022 Winter Olympics',
   'discipline': 'Short Track Speed Skating (Skating)',
   'noc': 'KOR'}]}

In [63]:
data[144878]

{'id': 148605,
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148605.jpg',
 'Roles': 'Competed in Olympic Games • Other',
 'Sex': 'Male',
 'Full_name': 'Fayik•Abdi',
 'Used_name': 'Fayik•Abdi',
 'Original_name': 'فايق•عابدي',
 'Born': '7 October 1997 in San Diego, California (USA)',
 'NOC': 'Kingdom of Saudi Arabia',
 'games': [{'game': '2022 Winter Olympics',
   'discipline': 'Alpine Skiing (Skiing)',
   'noc': 'KSA'}]}

In [62]:
data[145479]

IndexError: list index out of range

In [123]:
df = pd.DataFrame(data[0])

temp = pd.json_normalize(data[0], record_path=["games"])
# temp2 = pd.json_normalize(temp, record_path=["Events"])

In [124]:
Final = pd.concat([df.drop(['games'], axis = 1), temp], axis = 1)
Final

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,NOC,game,discipline,noc
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,Republic of Korea,2022 Winter Olympics,Short Track Speed Skating (Skating),KOR


On flatten le JSON, pour commencer, on flatten les jeux auquel un joueur a participé, cela va augmenter le nombre de lignes dans la table. Les événements des jeux, sont encore imbriqués dans une liste.

In [125]:
def unwind_games(data: list):
    df = pd.DataFrame(data[0])
    temp = pd.json_normalize(data[0], record_path=["games"])
    Final = pd.concat([df.drop(['games'], axis = 1), temp], axis = 1)
    for player in data[1:]:
        df2 = pd.DataFrame(player)
        temp2 = pd.json_normalize(player, record_path=["games"])
        new_df = pd.concat([df2.drop(['games'], axis = 1), temp2], axis = 1)
        Final = pd.concat([Final, new_df], ignore_index=True, sort=False)
        print(len(Final))
    return Final

In [126]:
Final = unwind_games(data)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
269
270
271
272
273
274
275
276
277
278
27

In [127]:
Final

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,NOC,game,discipline,noc,Affiliations,Measurements,Events,Nick/petnames,Died
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,Republic of Korea,2022 Winter Olympics,Short Track Speed Skating (Skating),KOR,NaN,NaN,NaN,NaN,NaN
1,148602,https://d2a3o6pzho379u.cloudfront.net/148602.jpg,Competed in Olympic Games,Male,Lee•Jun-Seo,Lee•Jun-Seo,Oriental,이•준서,Lee June-Seo,3 June 2000,Republic of Korea,2022 Winter Olympics,Short Track Speed Skating (Skating),KOR,NaN,NaN,NaN,NaN,NaN
2,148603,https://d2a3o6pzho379u.cloudfront.net/148603.jpg,Competed in Olympic Games,Male,Park•Jang-Hyeok,Park•Jang-Hyeok,Oriental,박•장혁,Park Jang-Hyuk,31 October 1998,Republic of Korea,2022 Winter Olympics,Short Track Speed Skating (Skating),KOR,NaN,NaN,NaN,NaN,NaN
3,148604,NaN,Competed in Olympic Games • Other,Female,Kiana•Kryeziu,Kiana•Kryeziu,NaN,NaN,NaN,11 November 2004,Kosovo,2022 Winter Olympics,Alpine Skiing (Skiing),KOS,NaN,NaN,NaN,NaN,NaN
4,148605,https://d2a3o6pzho379u.cloudfront.net/148605.jpg,Competed in Olympic Games • Other,Male,Fayik•Abdi,Fayik•Abdi,NaN,فايق•عابدي,NaN,"7 October 1997 in San Diego, California (USA)",Kingdom of Saudi Arabia,2022 Winter Olympics,Alpine Skiing (Skiing),KSA,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,149222,NaN,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,NaN,NaN,NaN,"30 January 2002 in Serov, Sverdlovsk (RUS)",ROC,2022 Winter Olympics,Ice Hockey (Ice Hockey),ROC,"Agidel, Ufa (RUS)",167 cm / 61 kg,"[{'event': 'Ice Hockey, Women(Olympic)', 'team...",NaN,NaN
603,149223,NaN,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,NaN,Валерия Сергеевна•Меркушева,NaN,"20 September 1999 in Moskva (Moscow), Moskva (...",ROC,2022 Winter Olympics,Ice Hockey (Ice Hockey),ROC,"SKIF Nizhny Novgorod, Nizhny Novgorod (RUS)",168 cm / 65 kg,"[{'event': 'Ice Hockey, Women(Olympic)', 'team...",NaN,NaN
604,149224,NaN,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,NaN,Юлия•Смирнова,NaN,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",ROC,2022 Winter Olympics,Ice Hockey (Ice Hockey),ROC,"Agidel, Ufa (RUS)",163 cm / 55 kg,"[{'event': 'Ice Hockey, Women(Olympic)', 'team...",NaN,NaN
605,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,NaN,NaN,NaN,"19 May 1899 in Niort, Deux-Sèvres (FRA)",France,1924 Summer Olympics,Athletics,FRA,"Beauvoisine FC, Rouen (FRA)",166 cm,"[{'event': '400 metres Hurdles, Men(Olympic)',...",NaN,"18 March 1986 in Niort, Deux-Sèvres (FRA)"


In [128]:
Final.to_json("final2.json", orient="records")

Le JSON final est trop grand pour le push sur la repos GitHub, on va donc diviser le fichier JSON en deux

In [129]:
with open("final2.json", 'r') as f:
    data = json.load(f)

In [7]:
len(data)/2

104060.0

In [12]:
len(data[:104060])

104060

In [15]:
len(data[104060:])

104060

On sauvegarde les deux fichiers et on push

In [16]:
# with open("final_part1.json", "w") as f:
#     json.dump(data[:104060], f)
# with open("final_part2.json", "w") as f:
#     json.dump(data[104060:], f)

In [22]:
pd.json_normalize(data[44452], record_path=["Events"])

,event,team,position,medal
0,"Handball, Women(Olympic)",Yugoslavia,4,


In [92]:
data[0]['id']

148601

Cette fonction va flatten les événements qui sont encore imbriqués, on procède de la même manière sauf qu'on ajoute des conditions. Un joueur a forcement participé à un jeux, mais les événements auquels il a participé peuvent ne pas être listés. Ce qui va engeindrer une erreur si on ne traite pas cette condition.

In [113]:
def unwind_events(data: list):
    if data[0]["Events"] == None:
        df = pd.DataFrame(data[0], index=[0])
        Final = df
    else:
        df = pd.DataFrame(data[0])
        temp = pd.json_normalize(data[0], record_path=["Events"])
        Final = pd.concat([df.drop(['Events'], axis = 1), temp], axis = 1)
    for player in data[1:]:
        if player["Events"] != None:
            df2 = pd.DataFrame(player)
            temp2 = pd.json_normalize(player, record_path=["Events"])
            new_df = pd.concat([df2.drop(['Events'], axis = 1), temp2], axis = 1)
            Final = pd.concat([Final, new_df], ignore_index=True, sort=False)
        else:
            df2 = pd.DataFrame(player, index=[0])
            Final = pd.concat([Final, df2], ignore_index=True, sort=False)
        print(player["id"])
    return Final

La fonction prends beaucoup de temps

In [130]:
final = unwind_events(data)

148602
148603
148604
148605
148606
148607
148608
148609
148610
148611
148612
148613
148614
148615
148616
148618
148620
148621
148622
148623
148624
148625
148626
148627
148628
148629
148630
148631
148632
148633
148634
148635
148636
148637
148638
148639
148640
148641
148642
148643
148644
148645
148646
148647
148648
148649
148650
148651
148652
148654
148655
148656
148657
148659
148660
148661
148662
148663
148664
148665
148666
148667
148668
148669
148670
148671
148672
148673
148674
148675
148676
148677
148677
148678
148679
148680
148681
148682
148683
148684
148685
148686
148687
148688
148689
148690
148691
148692
148693
148695
148696
148697
148699
148700
148701
148702
148703
148704
148705
148706
148707
148708
148709
148710
148711
148712
148713
148714
148715
148716
148717
148718
148719
148720
148721
148722
148723
148724
148725
148726
148727
148728
148729
148731
148732
148733
148734
148735
148736
148737
148738
148739
148740
148741
148742
148743
148744
148745
148746
148747
148748
148749
148750

In [131]:
final.to_json("events2.json", orient="records")

In [132]:
final

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,...,noc,Affiliations,Measurements,Events,Nick/petnames,Died,event,team,position,medal
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,...,KOR,None,None,None,None,None,NaN,NaN,NaN,NaN
1,148602,https://d2a3o6pzho379u.cloudfront.net/148602.jpg,Competed in Olympic Games,Male,Lee•Jun-Seo,Lee•Jun-Seo,Oriental,이•준서,Lee June-Seo,3 June 2000,...,KOR,None,None,None,None,None,NaN,NaN,NaN,NaN
2,148603,https://d2a3o6pzho379u.cloudfront.net/148603.jpg,Competed in Olympic Games,Male,Park•Jang-Hyeok,Park•Jang-Hyeok,Oriental,박•장혁,Park Jang-Hyuk,31 October 1998,...,KOR,None,None,None,None,None,NaN,NaN,NaN,NaN
3,148604,None,Competed in Olympic Games • Other,Female,Kiana•Kryeziu,Kiana•Kryeziu,None,None,None,11 November 2004,...,KOS,None,None,None,None,None,NaN,NaN,NaN,NaN
4,148605,https://d2a3o6pzho379u.cloudfront.net/148605.jpg,Competed in Olympic Games • Other,Male,Fayik•Abdi,Fayik•Abdi,None,فايق•عابدي,None,"7 October 1997 in San Diego, California (USA)",...,KSA,None,None,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,149222,None,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,None,None,None,"30 January 2002 in Serov, Sverdlovsk (RUS)",...,ROC,"Agidel, Ufa (RUS)",167 cm / 61 kg,None,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
925,149223,None,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,None,Валерия Сергеевна•Меркушева,None,"20 September 1999 in Moskva (Moscow), Moskva (...",...,ROC,"SKIF Nizhny Novgorod, Nizhny Novgorod (RUS)",168 cm / 65 kg,None,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
926,149224,None,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,None,Юлия•Смирнова,None,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",...,ROC,"Agidel, Ufa (RUS)",163 cm / 55 kg,None,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
927,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,None,None,None,"19 May 1899 in Niort, Deux-Sèvres (FRA)",...,FRA,"Beauvoisine FC, Rouen (FRA)",166 cm,None,None,"18 March 1986 in Niort, Deux-Sèvres (FRA)","400 metres Hurdles, Men(Olympic)",,5 h5 r1/3,


In [134]:
len(final)

929

In [ ]:
final.to_json("final2.json", orient="records")

In [5]:
with open("final_part2.json", "r") as f:
    rest_data = json.load(f)

with open("events.json", "r") as f:
    event_data = json.load(f)

In [11]:
rest_data

[{'id': 75363,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/75363.jpg',
  'Roles': 'Competed in Olympic Games',
  'Sex': 'Female',
  'Full_name': 'Irina Nikolayevna•Korzhanenko',
  'Used_name': 'Irina•Korzhanenko',
  'Born': '16 May 1974 in Azov, Rostov (RUS)',
  'Died': None,
  'NOC': 'Russian Federation',
  'game': '2004 Summer Olympics',
  'discipline': 'Athletics',
  'noc': 'RUS',
  'Events': [{'event': 'Shot Put, Women(Olympic)',
    'team': '',
    'position': 'DQ',
    'medal': ''}],
  'Measurements': '178 cm / 98 kg',
  'Affiliations': 'Rostov-na-Donu VS/Rostov-na-Donu Army',
  'Nick/petnames': None,
  'Title(s)': None,
  'Other_names': None,
  'Nationality': None,
  'Original_name': 'Ирина Николаевна•Коржаненко',
  'Name_order': None},
 {'id': 75364,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/75364.jpg',
  'Roles': 'Competed in Olympic Games • Non-starter',
  'Sex': 'Female',
  'Full_name': 'Olga Ivanovna•Kotlyarova',
  'Used_name': 'Olga•Kotlyarova',
  '

In [10]:
event_data[-1]

{'id': 148600,
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148600.jpg',
 'Roles': 'Non-starter',
 'Sex': 'Female',
 'Full_name': 'Park•Ji-Yun',
 'Used_name': 'Park•Ji-Yun',
 'Born': '22 July 1999',
 'Died': None,
 'NOC': 'Republic of Korea',
 'game': '2022 Winter Olympics',
 'discipline': 'Short Track Speed Skating (Skating)',
 'noc': 'KOR',
 'Measurements': None,
 'Affiliations': None,
 'Nick/petnames': None,
 'Title(s)': None,
 'Other_names': None,
 'Nationality': None,
 'Original_name': '박•지윤',
 'Name_order': 'Oriental',
 'event': '3,000 metres Relay, Women(Olympic)',
 'team': 'Republic of Korea',
 'position': 'DNS',
 'medal': ''}

In [45]:
df = pd.read_json("final_part2.json")
df[df['id'] == 148600]

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,noc,Events,Measurements,Affiliations,Nick/petnames,Title(s),Other_names,Nationality,Original_name,Name_order
103452,148600,https://d2a3o6pzho379u.cloudfront.net/148600.jpg,Non-starter,Female,Park•Ji-Yun,Park•Ji-Yun,22 July 1999,None,Republic of Korea,2022 Winter Olympics,...,KOR,"[{'event': '3,000 metres Relay, Women(Olympic)...",None,None,None,None,None,None,박•지윤,Oriental


Ici, on se rend compte que sur le site il y a une partie des joueurs qui sont listés mais qui n'ont pas d'évenements, ce qui a causé une erreur dans le programme. On va donc chercher à gérer cette condition dans la fonction plus haut.

In [46]:
df[df['id'] == 148601]

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,noc,Events,Measurements,Affiliations,Nick/petnames,Title(s),Other_names,Nationality,Original_name,Name_order
103453,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,23 April 1993,None,Republic of Korea,2022 Winter Olympics,...,KOR,None,None,None,None,None,Kim Dongwook,None,김•동욱,Oriental


In [22]:
data2 = rest_data[103453:]
data2

[{'id': 148601,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148601.jpg',
  'Roles': 'Competed in Olympic Games',
  'Sex': 'Male',
  'Full_name': 'Kim•Dong-Uk',
  'Used_name': 'Kim•Dong-Uk',
  'Born': '23 April 1993',
  'Died': None,
  'NOC': 'Republic of Korea',
  'game': '2022 Winter Olympics',
  'discipline': 'Short Track Speed Skating (Skating)',
  'noc': 'KOR',
  'Events': None,
  'Measurements': None,
  'Affiliations': None,
  'Nick/petnames': None,
  'Title(s)': None,
  'Other_names': 'Kim Dongwook',
  'Nationality': None,
  'Original_name': '김•동욱',
  'Name_order': 'Oriental'},
 {'id': 148602,
  'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148602.jpg',
  'Roles': 'Competed in Olympic Games',
  'Sex': 'Male',
  'Full_name': 'Lee•Jun-Seo',
  'Used_name': 'Lee•Jun-Seo',
  'Born': '3 June 2000',
  'Died': None,
  'NOC': 'Republic of Korea',
  'game': '2022 Winter Olympics',
  'discipline': 'Short Track Speed Skating (Skating)',
  'noc': 'KOR',
  'Events': None,
  '

In [23]:
data2[0]

{'id': 148601,
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/148601.jpg',
 'Roles': 'Competed in Olympic Games',
 'Sex': 'Male',
 'Full_name': 'Kim•Dong-Uk',
 'Used_name': 'Kim•Dong-Uk',
 'Born': '23 April 1993',
 'Died': None,
 'NOC': 'Republic of Korea',
 'game': '2022 Winter Olympics',
 'discipline': 'Short Track Speed Skating (Skating)',
 'noc': 'KOR',
 'Events': None,
 'Measurements': None,
 'Affiliations': None,
 'Nick/petnames': None,
 'Title(s)': None,
 'Other_names': 'Kim Dongwook',
 'Nationality': None,
 'Original_name': '김•동욱',
 'Name_order': 'Oriental'}

In [41]:
pd.DataFrame(data2[0], index=[0])

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,noc,Events,Measurements,Affiliations,Nick/petnames,Title(s),Other_names,Nationality,Original_name,Name_order
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,23 April 1993,None,Republic of Korea,2022 Winter Olympics,...,KOR,None,None,None,None,None,Kim Dongwook,None,김•동욱,Oriental


In [34]:
pd.DataFrame(data2[0], index=[0]).Events[0] == None

True

In [40]:
data2[0]["Events"] == None

True

Voici le début de la fonction unwind_events. C'est ici qu'on a codé la condition pour ensuite l'implémenter dans la fonciton unwind_events

In [42]:
if data2[0]["Events"] == None:
    df = pd.DataFrame(data2[0], index=[0])
    Final = df
else:
    df = pd.DataFrame(data2[0])
    temp = pd.json_normalize(data2[0], record_path=["Events"])
    Final = pd.concat([df.drop(['Events'], axis = 1), temp], axis = 1)
for player in data2[1:]:
    if player["Events"] != None:
        df2 = pd.DataFrame(player)
        temp2 = pd.json_normalize(player, record_path=["Events"])
        new_df = pd.concat([df2.drop(['Events'], axis = 1), temp2], axis = 1)
        Final = pd.concat([Final, new_df], ignore_index=True, sort=False)
    else:
        df2 = pd.DataFrame(player, index=[0])
        Final = pd.concat([Final, df2], ignore_index=True, sort=False)
    print(player["id"])

148602
148603
148604
148605
148606
148607
148608
148609
148610
148611
148612
148613
148614
148615
148616
148618
148620
148621
148622
148623
148624
148625
148626
148627
148628
148629
148630
148631
148632
148633
148634
148635
148636
148637
148638
148639
148640
148641
148642
148643
148644
148645
148646
148647
148648
148649
148650
148651
148652
148654
148655
148656
148657
148659
148660
148661
148662
148663
148664
148665
148666
148667
148668
148669
148670
148671
148672
148673
148674
148675
148676
148677
148677
148678
148679
148680
148681
148682
148683
148684
148685
148686
148687
148688
148689
148690
148691
148692
148693
148695
148696
148697
148699
148700
148701
148702
148703
148704
148705
148706
148707
148708
148709
148710
148711
148712
148713
148714
148715
148716
148717
148718
148719
148720
148721
148722
148723
148724
148725
148726
148727
148728
148729
148731
148732
148733
148734
148735
148736
148737
148738
148739
148740
148741
148742
148743
148744
148745
148746
148747
148748
148749
148750

In [44]:
Final["Events"]

0      None
1      None
2      None
3      None
4      None
       ... 
924    None
925    None
926    None
927    None
928     NaN
Name: Events, Length: 929, dtype: object

In [135]:
events1 = pd.read_json("events.json")
events2 = pd.read_json("events2.json")

In [136]:
events1

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,Nick/petnames,Title(s),Other_names,Nationality,Original_name,Name_order,event,team,position,medal
0,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,None,"Singles, Men(Olympic)",,=17,
1,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,None,"Doubles, Men(Olympic)",Jean Montariol,DNS,
2,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,None,"Singles, Men(Olympic)",,=32,
3,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,None,"Doubles, Mixed(Olympic)",Jeanne Vaussard,=8,
4,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,None,"Doubles, Men(Olympic)",Jacques Brugnon,4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305962,148597,https://d2a3o6pzho379u.cloudfront.net/148597.jpg,Competed in Olympic Games,Male,Lee•Si-Hyeong,Lee•Si-Hyeong,15 December 2000,None,Republic of Korea,2022 Winter Olympics,...,None,None,None,None,이•시형,Oriental,"Singles, Men(Olympic)",,27 r1/2,
305963,148598,https://d2a3o6pzho379u.cloudfront.net/148598.jpg,Competed in Olympic Games,Female,Kim•Eun-Ji,Kim•Eun-Ji,27 March 1992,None,Republic of Korea,2022 Winter Olympics,...,None,None,None,None,김•은지,Oriental,"Skeleton, Women(Olympic)",,23,
305964,148599,https://d2a3o6pzho379u.cloudfront.net/148599.jpg,Competed in Olympic Games,Male,Park•Seong-Hyeon,Park•Seong-Hyeon,12 October 1999,None,Republic of Korea,2022 Winter Olympics,...,None,None,None,None,박•성현,Oriental,"1,500 metres, Men(Olympic)",,21,
305965,148599,https://d2a3o6pzho379u.cloudfront.net/148599.jpg,Competed in Olympic Games,Male,Park•Seong-Hyeon,Park•Seong-Hyeon,12 October 1999,None,Republic of Korea,2022 Winter Olympics,...,None,None,None,None,박•성현,Oriental,"Team Pursuit (8 laps), Men(Olympic)",Republic of Korea,6,


In [137]:
events2

,id,photo_link,Roles,Sex,Full_name,Used_name,Name_order,Original_name,Other_names,Born,...,noc,Affiliations,Measurements,Events,Nick/petnames,Died,event,team,position,medal
0,148601,https://d2a3o6pzho379u.cloudfront.net/148601.jpg,Competed in Olympic Games,Male,Kim•Dong-Uk,Kim•Dong-Uk,Oriental,김•동욱,Kim Dongwook,23 April 1993,...,KOR,None,None,NaN,None,None,None,None,None,None
1,148602,https://d2a3o6pzho379u.cloudfront.net/148602.jpg,Competed in Olympic Games,Male,Lee•Jun-Seo,Lee•Jun-Seo,Oriental,이•준서,Lee June-Seo,3 June 2000,...,KOR,None,None,NaN,None,None,None,None,None,None
2,148603,https://d2a3o6pzho379u.cloudfront.net/148603.jpg,Competed in Olympic Games,Male,Park•Jang-Hyeok,Park•Jang-Hyeok,Oriental,박•장혁,Park Jang-Hyuk,31 October 1998,...,KOR,None,None,NaN,None,None,None,None,None,None
3,148604,None,Competed in Olympic Games • Other,Female,Kiana•Kryeziu,Kiana•Kryeziu,None,None,None,11 November 2004,...,KOS,None,None,NaN,None,None,None,None,None,None
4,148605,https://d2a3o6pzho379u.cloudfront.net/148605.jpg,Competed in Olympic Games • Other,Male,Fayik•Abdi,Fayik•Abdi,None,فايق•عابدي,None,"7 October 1997 in San Diego, California (USA)",...,KSA,None,None,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,149222,None,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,None,None,None,"30 January 2002 in Serov, Sverdlovsk (RUS)",...,ROC,"Agidel, Ufa (RUS)",167 cm / 61 kg,NaN,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
925,149223,None,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,None,Валерия Сергеевна•Меркушева,None,"20 September 1999 in Moskva (Moscow), Moskva (...",...,ROC,"SKIF Nizhny Novgorod, Nizhny Novgorod (RUS)",168 cm / 65 kg,NaN,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
926,149224,None,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,None,Юлия•Смирнова,None,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",...,ROC,"Agidel, Ufa (RUS)",163 cm / 55 kg,NaN,None,None,"Ice Hockey, Women(Olympic)",ROC,5,
927,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,None,None,None,"19 May 1899 in Niort, Deux-Sèvres (FRA)",...,FRA,"Beauvoisine FC, Rouen (FRA)",166 cm,NaN,None,"18 March 1986 in Niort, Deux-Sèvres (FRA)","400 metres Hurdles, Men(Olympic)",,5 h5 r1/3,


In [138]:
events_total = pd.concat([events1, events2], ignore_index=True, sort=False)

In [139]:
events_total

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,Title(s),Other_names,Nationality,Original_name,Name_order,event,team,position,medal,Events
0,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,"Singles, Men(Olympic)",,=17,,NaN
1,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,"Doubles, Men(Olympic)",Jean Montariol,DNS,,NaN
2,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Singles, Men(Olympic)",,=32,,NaN
3,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Doubles, Mixed(Olympic)",Jeanne Vaussard,=8,,NaN
4,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Doubles, Men(Olympic)",Jacques Brugnon,4,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306891,149222,None,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,"30 January 2002 in Serov, Sverdlovsk (RUS)",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,None,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306892,149223,None,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,"20 September 1999 in Moskva (Moscow), Moskva (...",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,Валерия Сергеевна•Меркушева,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306893,149224,None,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,Юлия•Смирнова,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306894,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,"19 May 1899 in Niort, Deux-Sèvres (FRA)","18 March 1986 in Niort, Deux-Sèvres (FRA)",France,1924 Summer Olympics,...,NaN,None,NaN,None,None,"400 metres Hurdles, Men(Olympic)",,5 h5 r1/3,,NaN


On enregistre le JSON final et ensuite on le divise pour pouvoir le push sur le GitHub. On a mis ce document en dessous, dans le gitignore.

In [141]:
events_total.to_json("events_total.json", orient="records")

In [142]:
len(events_total)/2

153448.0

In [143]:
events_total.iloc[:153448]

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,Title(s),Other_names,Nationality,Original_name,Name_order,event,team,position,medal,Events
0,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,"Singles, Men(Olympic)",,=17,,NaN
1,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1912 Summer Olympics,...,None,None,None,None,None,"Doubles, Men(Olympic)",Jean Montariol,DNS,,NaN
2,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Singles, Men(Olympic)",,=32,,NaN
3,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Doubles, Mixed(Olympic)",Jeanne Vaussard,=8,,NaN
4,1,https://d2a3o6pzho379u.cloudfront.net/1.jpg,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1920 Summer Olympics,...,None,None,None,None,None,"Doubles, Men(Olympic)",Jacques Brugnon,4,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153443,73859,None,Competed in Olympic Games,Male,"Johannes Michaël ""Jan""•Meijer",Jan•Meijer,"8 June 1921 in Amsterdam, Noord-Holland (NED)","21 October 1993 in Amsterdam, Noord-Holland (NED)",Netherlands,1948 Summer Olympics,...,None,None,None,None,None,"100 metres, Men(Olympic)",,3 h4 r1/4,,NaN
153444,73859,None,Competed in Olympic Games,Male,"Johannes Michaël ""Jan""•Meijer",Jan•Meijer,"8 June 1921 in Amsterdam, Noord-Holland (NED)","21 October 1993 in Amsterdam, Noord-Holland (NED)",Netherlands,1948 Summer Olympics,...,None,None,None,None,None,"4 × 100 metres Relay, Men(Olympic)",Netherlands,6,,NaN
153445,73860,https://d2a3o6pzho379u.cloudfront.net/73860.jpg,Competed in Olympic Games,Male,Emiel Remco•Mellaard,Emiel•Mellaard,"21 March 1966 in Spijkenisse, Zuid-Holland (NED)",None,Netherlands,1988 Summer Olympics,...,None,None,None,None,None,"Long Jump, Men(Olympic)",,11,,NaN
153446,73861,None,Competed in Olympic Games,Male,Menso Johannes•Menso,Menso•Menso,"14 April 1903 in Den Haag (The Hague), Zuid-Ho...","26 April 1990 in Amsterdam, Noord-Holland (NED)",Netherlands,1924 Summer Olympics,...,None,None,None,None,None,"400 metres, Men(Olympic)",,4 h9 r1/4,,NaN


In [144]:
events_total.iloc[153448:]

,id,photo_link,Roles,Sex,Full_name,Used_name,Born,Died,NOC,game,...,Title(s),Other_names,Nationality,Original_name,Name_order,event,team,position,medal,Events
153448,73861,None,Competed in Olympic Games,Male,Menso Johannes•Menso,Menso•Menso,"14 April 1903 in Den Haag (The Hague), Zuid-Ho...","26 April 1990 in Amsterdam, Noord-Holland (NED)",Netherlands,1924 Summer Olympics,...,None,None,None,None,None,"4 × 400 metres Relay, Men(Olympic)",Netherlands,DNS,,NaN
153449,73862,https://d2a3o6pzho379u.cloudfront.net/73862.jpg,Competed in Olympic Games,Male,"Gerardus Hermanus Maria ""Gerard""•Nijboer",Gerard•Nijboer,"18 August 1955 in Hasselt, Overijssel (NED)",None,Netherlands,1980 Summer Olympics,...,None,None,None,None,None,"Marathon, Men(Olympic)",,2,Silver,NaN
153450,73862,https://d2a3o6pzho379u.cloudfront.net/73862.jpg,Competed in Olympic Games,Male,"Gerardus Hermanus Maria ""Gerard""•Nijboer",Gerard•Nijboer,"18 August 1955 in Hasselt, Overijssel (NED)",None,Netherlands,1984 Summer Olympics,...,None,None,None,None,None,"Marathon, Men(Olympic)",,DNF,,NaN
153451,73862,https://d2a3o6pzho379u.cloudfront.net/73862.jpg,Competed in Olympic Games,Male,"Gerardus Hermanus Maria ""Gerard""•Nijboer",Gerard•Nijboer,"18 August 1955 in Hasselt, Overijssel (NED)",None,Netherlands,1988 Summer Olympics,...,None,None,None,None,None,"Marathon, Men(Olympic)",,13,,NaN
153452,73863,https://d2a3o6pzho379u.cloudfront.net/73863.jpg,Competed in Olympic Games,Male,"Martinus Bernardus ""Tinus""•Osendarp",Tinus•Osendarp,"21 May 1916 in Delft, Zuid-Holland (NED)","20 June 2002 in Heerlen, Limburg (NED)",Netherlands,1936 Summer Olympics,...,None,None,None,None,None,"100 metres, Men(Olympic)",,3,Bronze,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306891,149222,None,Competed in Olympic Games,Female,Polina•Luchnikova,Polina•Luchnikova,"30 January 2002 in Serov, Sverdlovsk (RUS)",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,None,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306892,149223,None,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya•Merkusheva,"20 September 1999 in Moskva (Moscow), Moskva (...",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,Валерия Сергеевна•Меркушева,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306893,149224,None,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya•Smirnova,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",None,ROC,2022 Winter Olympics,...,NaN,None,NaN,Юлия•Смирнова,None,"Ice Hockey, Women(Olympic)",ROC,5,,NaN
306894,149225,https://d2a3o6pzho379u.cloudfront.net/149225.jpg,Competed in Olympic Games,Male,André•Foussard,André•Foussard,"19 May 1899 in Niort, Deux-Sèvres (FRA)","18 March 1986 in Niort, Deux-Sèvres (FRA)",France,1924 Summer Olympics,...,NaN,None,NaN,None,None,"400 metres Hurdles, Men(Olympic)",,5 h5 r1/3,,NaN


Sauvegarde des deux fichiers

In [145]:
# events_total.iloc[:153448].to_json("final_events_1.json", orient="records")
# events_total.iloc[153448:].to_json("final_events_2.json", orient="records")